In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio
sns.set()

## Load files

In [ ]:
# loading files from allen file system mat directory
# mount_point    = '/Users/fruity/Remote-AI-root/allen/aibs' #Rohan's mount point
# mount_point    = '/allen/aibs' #Corinne's mount point
# data_dir       = mount_point + 'mat/Corinne/autoencoder/'
#embeddings_dir = mount_point+'mat/Corinne/autoencoder/embeddings/'

#Load Corinne's data file:
#dat_df=pd.read_csv(data_dir+'ae_data_50hz_sequential_10_24_2019.csv', sep='#',low_memory=False)

#Load embeddings:
#mat = sio.loadmat(embeddings_dir+'TEST_cv_0_pd_0-2_bs_1000_ld_2_ne_10000_ri_0-summary.mat, \
#                  squeeze_me=True)

In [ ]:
# loading files locally
dat_df=pd.read_csv('ae_data_50hz_sequential_10_24_2019.csv', sep='#',low_memory=False)
Z = sio.loadmat('TEST_cv_0_pd_0-2_bs_1000_ld_2_ne_10000_ri_0-summary.mat', squeeze_me=True)
dat_df.keys()

In [ ]:
#Separate out the annotations:
anno_df = dat_df[dat_df.keys().values[0:11]]
print('anno_df: ', anno_df.columns.values)

In [ ]:
# make some new catagories to deal with unknown cre_lines
anno_df['pre_cat']=anno_df['pre_cre']
anno_df['post_cat']=anno_df['post_cre']
anno_df['pre_cat'].loc[anno_df['species'] == 'human'] = 'human'
anno_df['post_cat'].loc[anno_df['species'] == 'human'] = 'human'

anno_df['pre_cat'].loc[(anno_df['pre_cre'] == 'unknown') & (anno_df['species'] == 'mouse') & (anno_df['pre_layer'] == '2/3')] = '2/3'
anno_df['post_cat'].loc[(anno_df['post_cre'] == 'unknown') & (anno_df['species'] == 'mouse') & (anno_df['post_layer'] == '2/3')] = '2/3'

anno_df['pre_cat'].loc[(anno_df['pre_cre'] == 'unknown') & (anno_df['species'] == 'mouse') & (anno_df['pre_layer'] == '4')] = '4'
anno_df['post_cat'].loc[(anno_df['post_cre'] == 'unknown') & (anno_df['species'] == 'mouse') & (anno_df['post_layer'] == '4')] = '4'

anno_df['pre_cat'].loc[(anno_df['pre_cre'] == 'unknown') & (anno_df['species'] == 'mouse') & (anno_df['pre_layer'] == '6')] = '6'
anno_df['post_cat'].loc[(anno_df['post_cre'] == 'unknown') & (anno_df['species'] == 'mouse') & (anno_df['post_layer'] == '6')] = '6'

anno_df['pre_cat'].loc[(anno_df['pre_cre'] == 'unknown') & (anno_df['species'] == 'mouse') & (anno_df['pre_layer'] == '5')] = '5'
anno_df['post_cat'].loc[(anno_df['post_cre'] == 'unknown') & (anno_df['species'] == 'mouse') & (anno_df['post_layer'] == '5')] = '5'

In [ ]:
#Annotation combinations
anno_df = anno_df.assign(pre_post_cat=(anno_df['pre_cat'].map(str) + '_' + anno_df['post_cat'].map(str)))
anno_df = anno_df.assign(pre_post_ex=(anno_df['pre_ex'].map(str) + '_' + anno_df['post_ex'].map(str)))

#Assign numeric id to 'combined_anno'
anno_df = anno_df.assign(num_pre_post_cat=(anno_df['pre_post_cat']).astype('category').cat.codes)
anno_df = anno_df.assign(num_pre_post_ex=(anno_df['pre_post_ex']).astype('category').cat.codes)

In [ ]:
anno_df.keys()

In [ ]:
Z['zY']

In [ ]:
plt.figure(figsize = (10,10))
plt.scatter(Z['zY'][:,0], Z['zY'][:,1],s=1,c=anno_df['num_pre_post_ex'].values, cmap='viridis')
plt.legend()
ax = plt.gca()
ax.set_xlim(-7,7)
ax.set_ylim(-7,7)
print(np.cov(np.transpose(Z['zX'])))

In [ ]:
# this doesn't work yet
import plotly
import plotly.graph_objs as go

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

# Configure the trace.
trace = go.Scatter3d(
#    x=[1, 2, 3],  # <-- Put your data instead
#    y=[4, 5, 6],  # <-- Put your data instead
#    z=[7, 8, 9],  # <-- Put your data instead
    x=Z['zX'][:,0], 
    y=Z['zX'][:,1],
    z=Z['zX'][:,2],
    s=1,
    c=anno_df['num_pre_post_ex'].values, 
    cmap='viridis',
    mode='markers',
    marker={
        'size': 10,
        'opacity': 0.8,
    }
)

# Configure the layout.
layout = go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0}
)

data = [trace]

plot_figure = go.Figure(data=data, layout=layout)

# Render the plot.
plotly.offline.iplot(plot_figure)